In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt

In [2]:


races_files=['Afrikan_american','American_Indian_Alaska_Native','Asian_Pacific_Islander','white']


In [3]:
44.0/273831

0.00016068304903389316

In [4]:
def get_df(i):
    df=pd.io.parsers.read_csv('raw_Data/%s.txt'%(races_files[i]), sep=r"\s+",quotechar='"',skiprows=[1])
    df=df[~np.all(df.ix[:,1:].isnull(),axis=1)]
    cols=[elt for elt in df.columns if elt not in ['Rate']]
    df=df[cols]
    df=df.rename(columns={'Crude':'Rate'})

    cols=[elt for elt in df.columns if elt not in ['Cancer Sites Code','Age Group','Sex Code','Race Code','Year Code']]
    df=df[cols]
    return df


In [5]:
pieces = [get_df(i) for i in range(len(races_files))]
concatenated = pd.concat(pieces)
concatenated.to_csv('all_races.csv')
concatenated.shape

(20607, 9)

In [9]:
concatenated['Age Group Code'].unique()

array(['1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85+'], dtype=object)

In [54]:
concatenated[1000:]
concatenated.Race.unique()

myfilter=concatenated.Race=='American Indian or Alaska Native'
myfilter2=concatenated['Leading Cancer Sites']=='Melanoma of the Skin'
concatenated[myfilter & myfilter2]

,Leading Cancer Sites,Leading Cancer Sites Code,Age Group Code,Sex,Race,Year,Count,Population,Rate
1226,Melanoma of the Skin,25010,55-59,Male,American Indian or Alaska Native,2006,16,80263,19.9


In [55]:
myfilter=concatenated.Race=='American Indian or Alaska Native'
myfilter2=concatenated['Leading Cancer Sites']=='Colon and Rectum'
myfilter3=concatenated['Year']==2006
concatenated[(myfilter & myfilter2) & myfilter3]

,Leading Cancer Sites,Leading Cancer Sites Code,Age Group Code,Sex,Race,Year,Count,Population,Rate
574,Colon and Rectum,21041-21052,40-44,Male,American Indian or Alaska Native,2006,17,132238,12.9
583,Colon and Rectum,21041-21052,45-49,Female,American Indian or Alaska Native,2006,21,128628,16.3
594,Colon and Rectum,21041-21052,45-49,Male,American Indian or Alaska Native,2006,35,122889,28.5
607,Colon and Rectum,21041-21052,50-54,Female,American Indian or Alaska Native,2006,40,108199,37.0
620,Colon and Rectum,21041-21052,50-54,Male,American Indian or Alaska Native,2006,40,101548,39.4
633,Colon and Rectum,21041-21052,55-59,Female,American Indian or Alaska Native,2006,36,85312,42.2
646,Colon and Rectum,21041-21052,55-59,Male,American Indian or Alaska Native,2006,47,80263,58.6
659,Colon and Rectum,21041-21052,60-64,Female,American Indian or Alaska Native,2006,34,56589,60.1
672,Colon and Rectum,21041-21052,60-64,Male,American Indian or Alaska Native,2006,46,52463,87.7
685,Colon and Rectum,21041-21052,65-69,Female,American Indian or Alaska Native,2006,42,40658,103.3


In [56]:
df_other=pd.io.parsers.read_csv('raw_Data/%s.txt'%('other'), sep=r"\s+",quotechar='"',skiprows=[1])
df_other=df_other[~np.all(df_other.ix[:,1:].isnull(),axis=1)]

In [57]:
df_other=df_other[['Age Group','Sex','Year','Year Code']]
df_other=df_other.rename(columns={'Year Code':'Count'})

In [58]:
df_other.to_csv('other_unknown.csv')

In [59]:
concatenated[:2]

,Leading Cancer Sites,Leading Cancer Sites Code,Age Group Code,Sex,Race,Year,Count,Population,Rate
0,All Sites Combined,00,1,Female,Black or African American,2000,44,273831,16.1
1,All Sites Combined,00,1,Female,Black or African American,2001,48,273501,17.6


In [111]:
df_1=concatenated.groupby(['Leading Cancer Sites','Sex']).apply(lambda g: int(np.sum(g['Count'])))
df_1=pd.DataFrame(df_1).reset_index()
df_1.rename(columns={'Leading Cancer Sites': 'category', 0:'measure'}, inplace=True)
female_filter=df_1.Sex=='Female'
df_female=df_1[female_filter]
df_male=df_1[~female_filter]
all_cancers=df_1.category.unique()
merged=pd.merge(df_female,df_male,on=['category'],how='outer')

merged['measure_y']=np.where(pd.isnull(merged.Sex_y),0.1,merged.measure_y)
merged['Sex_y'].fillna('Male',inplace='True')

merged['measure_x']=np.where(pd.isnull(merged.Sex_x),0.1,merged.measure_x)
merged['Sex_x'].fillna('Female',inplace='True')
merged['sum_column']=merged.measure_x+merged.measure_y
df_female=merged[['category','Sex_x','measure_x','sum_column']]
df_female.rename(columns={'Sex_x': 'Sex', 'measure_x':'measure'}, inplace=True)

df_male=merged[['category','Sex_y','measure_y','sum_column']]
df_male.rename(columns={'Sex_y': 'Sex', 'measure_y':'measure'}, inplace=True)
df_1=pd.concat([df_female,df_male])
myfilter=df_1.category=='All Sites Combined'
df_1=df_1[~myfilter]
df_1.set_index(np.arange(df_1.shape[0]),inplace='True')
# df_1


In [112]:

df_1.to_csv('first_bar_project.csv')
df_1.T.to_json('df_1_project.json')

In [94]:
df_2=concatenated.groupby(['Leading Cancer Sites','Race','Year','Sex'])\
.apply(lambda g: np.sum(g['Rate']*g['Population'])*1.0/np.sum(g['Population'])   )

df_2=pd.DataFrame(df_2)
df_2.reset_index(inplace=True)
df_2.rename(columns={'Leading Cancer Sites': 'group', 'Year':'category',0:'measure'}, inplace=True)
df_2.loc[df_2['group']=='All Sites Combined','group']='All'
df_2.T.to_json('df_2_project.json')

In [96]:
df_2.T

,0,1,2,3,4,5,6,7,8,9,...,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788
group,All,All,All,All,All,All,All,All,All,All,...,"Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ","Urinary Bladder, invasive and in situ"
Race,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,American Indian or Alaska Native,...,White,White,White,White,White,White,White,White,White,White
category,1999,1999,2000,2000,2001,2001,2002,2002,2003,2003,...,2007,2007,2008,2008,2009,2009,2010,2010,2011,2011
Sex,Female,Male,Female,Male,Female,Male,Female,Male,Female,Male,...,Female,Male,Female,Male,Female,Male,Female,Male,Female,Male
measure,240.8441,240.1654,265.277,259.7861,250.1131,235.4218,239.7931,216.6788,246.8939,242.897,...,18.07151,53.59136,17.95283,53.90941,17.8875,53.99055,18.96111,53.50207,17.52661,52.65115


In [14]:
plt.plot(df['category'],df['measure'])
plt.show()

NameError: name 'df' is not defined

In [15]:
df=df_2[df_2.Race=='Black or African American']

In [16]:
df=df[df.group=='Breast']

In [17]:
df_2['Race'].unique()

array(['American Indian or Alaska Native', 'Asian or Pacific Islander',
       'Black or African American', 'White'], dtype=object)

In [46]:
df_3=concatenated.groupby(['Age Group Code','Race','Leading Cancer Sites','Sex'])\
.apply(lambda g: np.sum(g['Rate']*g['Population'])*1.0/np.sum(g['Population'])   )
df_3=pd.DataFrame(df_3).reset_index()
df_3.rename(columns={0: 'rate'}, inplace=True)


In [47]:
myfilter=concatenated['Leading Cancer Sites']=='All Sites Combined'
temp=concatenated[myfilter].groupby(['Race','Sex','Age Group Code'])\
.apply(lambda g: np.mean(g['Population'])   )
temp=pd.DataFrame(temp).reset_index()
temp.rename(columns={0: 'population'}, inplace=True)
df_3=pd.merge(df_3,temp,on=['Race','Sex','Age Group Code'],how='left')


In [48]:


df_3.loc[df_3['Leading Cancer Sites']=='All Sites Combined','Leading Cancer Sites']='All'


df_3['Age Group Code'].unique()

age_dict={'1':1,'1-4':3 ,'10-14':12,'15-19':17,'20-24':22,'25-29':27,'30-34':32,\
         '35-39':37,'40-44':42,'45-49':47,'5-9':7,'50-54':52,'55-59':57,'60-64':62,\
         '65-69':67,'70-74':72,'75-79':77,'80-84':82,'85+':87}
df_3['age_num']=df_3['Age Group Code'].map(lambda x:np.float(age_dict[x]))


df_3.rename(columns={'Leading Cancer Sites': 'group', 'age_num':'category','rate':'measure'}, inplace=True)
df_3.T.to_json('df_3_project.json')
df_3.to_csv('df_3_project.csv')

In [24]:
# plt.plot(df_3.population)
# plt.show()
np.min(df_3.population)

6962.0

In [35]:

np.sort(df_3[df_3.group=='Leukemias']['measure'])

array([   1.70645595,    1.83080551,    1.93456806,    1.96870093,
          2.07856069,    2.17601566,    2.24070008,    2.28183733,
          2.33060892,    2.34071669,    2.65903985,    2.67453033,
          2.67600372,    2.69986996,    2.71343859,    2.72756662,
          2.7733188 ,    2.8       ,    2.80792938,    2.82284564,
          2.91843338,    2.93146228,    3.09105382,    3.11107258,
          3.15082554,    3.22800159,    3.30280845,    3.31990654,
          3.39258541,    3.39694065,    3.45524116,    3.48462625,
          3.66619557,    3.78944587,    3.89443178,    3.96940546,
          3.98508185,    3.98721235,    4.12844229,    4.17273214,
          4.18228196,    4.25335096,    4.36917309,    4.5409421 ,
          4.58247326,    4.59664437,    4.65192401,    4.74022435,
          4.84194488,    5.39451502,    5.4917017 ,    5.59950109,
          5.86976469,    6.00990536,    6.48150082,    6.83419019,
          7.26742285,    7.31168942,    7.97809811,    8.05174

In [64]:
plt.plot(df_3['age_num'],df_3[0])
plt.show()
    
    

In [50]:
plt.plot(df_3.race_num,df_3.age_num,marker='o')
plt.show()

In [46]:
df_3

,Age Group Code,Race,0,age,age_num,race_num
0,1,Asian or Pacific Islander,20.971523,0,0,None
1,1,Black or African American,17.769008,0,0,None
2,1,White,8.074469,0,0,None
3,1-4,American Indian or Alaska Native,15.361175,1,1,None
4,1-4,Asian or Pacific Islander,11.878298,1,1,None
5,1-4,Black or African American,6.340418,1,1,None
6,1-4,White,6.725561,1,1,None
7,10-14,American Indian or Alaska Native,9.319079,2,2,None
8,10-14,Asian or Pacific Islander,8.658492,2,2,None
9,10-14,Black or African American,4.498354,2,2,None
